# Fine-Tune FLAN-T5 with Reinforcement Learning (PPO) and PEFT to Generate Confidential Prompts

In [1]:
import os

%pip install torch torchdata  --index-url https://download.pytorch.org/whl/cu118 --quiet

%pip install transformers datasets evaluate rouge_score peft trl --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM, GenerationConfig, pipeline
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl: Transformer Reinforcement Learning library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import evaluate

import numpy as np
import pandas as pd

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

torch.cuda.is_available()

C:\Users\suppo\MLENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
DATASET = "DIBT/10k_prompts_ranked"

PEFT_CHECKTPOINT = f"./model_checkpoint"

ORIGINAL_MODEL_NAME = 'google/flan-t5-small'

REWARD_MODEL_NAME = "dslim/distilbert-NER"

## 2 - Load FLAN-T5 Model, Reward Model

### 2.1 - Load Data and FLAN-T5 Model Fine-Tuned with Confidential Prompts as autoencoder

In [4]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

original_model = AutoModelForSeq2SeqLM.from_pretrained(ORIGINAL_MODEL_NAME, torch_dtype=torch.bfloat16).to('cuda')

peft_model = PeftModel.from_pretrained(original_model, 
                                       PEFT_CHECKTPOINT, 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=True).to('cuda')

print(f'Peft fined tuned model:\n{print_number_of_trainable_model_parameters(peft_model)}\n')

ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,                                                               
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True).to('cuda')

tokenizer = AutoTokenizer.from_pretrained(ORIGINAL_MODEL_NAME, device_map="auto")

print(f'\nPPO model parameters to be updated:\n{print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)

ref_model = create_reference_model(ppo_model).to('cuda')

print(f'\nReference model PPO KL:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Peft fined tuned model:
trainable model parameters: 1376256
all model parameters: 78337408
percentage of trainable model parameters: 1.76%


PPO model parameters to be updated:
trainable model parameters: 1376769
all model parameters: 78337921
percentage of trainable model parameters: 1.76%

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=512, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

Reference model PPO KL:
trainable model parameters: 0
all model parameters: 78337921
percentage of trainable model parameters: 0.00%



In [5]:
dataset = load_dataset(DATASET, split='train')

dataset = dataset.filter(lambda x: len(x["prompt"]) > 50 and len(x["prompt"]) <= 300, batched=False)

dataset = dataset.train_test_split(test_size=0.1)

dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 4680
    })
    test: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 521
    })
})

In [6]:
def make_safe_prompt(prompt):
    return f"""Create a safe prompt from the following prompt:

{prompt}

Prompt:"""

def tokenize_function(sample):
    sample['input_ids'] = tokenizer.encode( make_safe_prompt(sample['prompt']) )

    sample['query'] = tokenizer.decode(sample['input_ids'])
    return sample

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=False)

tokenized_datasets = tokenized_datasets.remove_columns(['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', \
                                                        'raw_responses', 'kind', 'cluster_description', 'topic'])

tokenized_datasets

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 521/521 [00:00<00:00, 1943.59 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'query'],
        num_rows: 4680
    })
    test: Dataset({
        features: ['input_ids', 'query'],
        num_rows: 521
    })
})

### 2.2 - Reward Model

**Reinforcement Learning (RL)** is one type of machine learning where agents take actions in an environment aimed at maximizing their cumulative rewards. The agent's behavior is defined by the **policy**. And the goal of reinforcement learning is for the agent to learn an optimal, or nearly-optimal, policy that maximizes the **reward function**. 


In [7]:
reward_tokenizer = AutoTokenizer.from_pretrained(REWARD_MODEL_NAME)
reward_model = AutoModelForTokenClassification.from_pretrained(REWARD_MODEL_NAME).to('cuda')

ner_model = pipeline("ner", model=reward_model, tokenizer=reward_tokenizer, device="cuda")

def reward(data):
    ners = ner_model(data)
    score = sum(m['score'] for m in ners) / len(ners) if len(ners) else 0.0

    return {'label': 'ok' if score < 0.5 else 'protect', 'score': score}

test = "This is a private John's name. Generate a fake loan with it."
print(f'Results model: {ner_model(test)}')

print(f'Rewards: {reward(test)}')

Results model: [{'entity': 'B-PER', 'score': 0.7190356, 'index': 5, 'word': 'John', 'start': 18, 'end': 22}]
Rewards: {'label': 'protect', 'score': 0.7190356254577637}


In [8]:
def evaluator(model, ds, column):
    global score
    global count
    score = 0
    count = 0
    def evaluate(sample):
        global score
        global count
        r = model(sample[column])
        if r['label'] == 'protect':
            count += 1
            score += r['score']

        return sample

    ds.map(evaluate, batched=False)

    return count, score / count if count > 0 else 0

count, score = evaluator(reward, tokenized_datasets['train'], 'query')
print(f"Found {count} prompts to protect out of {len(tokenized_datasets['train'])}. Average score {score}")

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4680/4680 [00:40<00:00, 116.02 examples/s]

Found 1804 prompts to protect out of 4680. Average score 0.8473334156436817


In [18]:
def evaluate_ner_generation(model, size):

    generation_config = GenerationConfig(max_new_tokens=100, top_k=0.0, top_p=1.0, do_sample=True)

    scores = []
    for i, sample in tqdm(enumerate(tokenized_datasets['test'].select(range(size)))):
        prompt = sample["query"]
            
        inp = tokenizer(sample["query"], return_tensors="pt", padding=True).to('cuda').input_ids
        
        gen_ids = model.generate(input_ids=inp, generation_config=generation_config)
        
        generated_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
        
        r = reward(sample["query"] + " " + generated_text)

        scores.append( r['score'] )

    mean = np.mean(scores)
    std = np.std(scores)
        
    return mean, std

peft_mean, peft_std = evaluate_ner_generation(ref_model, size=len(tokenized_datasets['test']))

print(f"Average need protect score: {peft_mean} . Std: {peft_std}")

521it [07:35,  1.14it/s]

Average need protect score: 0.3070756154536768 . Std: 0.40759072981704814


<a name='3'></a>
## 3 - Perform Fine-Tuning to protect prompts
Optimize a RL policy against the reward model using Proximal Policy Optimization (PPO).

<a name='3.1'></a>
### 3.1 - Initialize `PPOTrainer`
 
For the `PPOTrainer` initialization, you will need a collator. Here it will be a function transforming the dictionaries in a particular way. You can define and test it:

Set up the configuration parameters. Load the `ppo_model` and the tokenizer. You will also load a frozen version of the model `ref_model`. The first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This works as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original LLM.

In [10]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

config = PPOConfig(
    model_name="ppo_model",    
    learning_rate=1.41e-5,
    ppo_epochs=1,
    mini_batch_size=1,
    batch_size=2
)

ppo_trainer = PPOTrainer(config=config, 
                         model=ppo_model, 
                         ref_model=ref_model, 
                         tokenizer=tokenizer, 
                         dataset=tokenized_datasets["train"], 
                         data_collator=collator)

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


<a name='3.2'></a>
### 3.2 - Fine-Tune the Model

The fine-tuning loop consists of the following main steps:
1. Get the query responses from the policy LLM (PEFT model).
2. Get reward values using distilbert-ner
3. Optimize policy with PPO using the (query, response, reward) triplet.

The operation is running if you see the following metrics appearing:
* `objective/kl`: minimize kl divergence,
* `ppo/returns/mean`: maximize mean returns,
* `ppo/policy/advantages_mean`: maximize advantages.

In [11]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):

    reply_tensors = []
    promp_tensors = []

    for prompt_ids in batch["input_ids"]:
        max_new_tokens = output_length_sampler()

        generation_kwargs["max_new_tokens"] = max_new_tokens

        inp = torch.as_tensor(prompt_ids).to('cuda')
        promp_tensors.append(inp)
        
        prompts = ppo_trainer.generate(inp, **generation_kwargs)
        
        reply_tensors.append(prompts.squeeze()[-max_new_tokens:])
        
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in reply_tensors]

    # Compute reward outputs.
    rewards = [reward(q + r) for q, r in zip(batch["query"], batch["response"])]
    reward_tensors = [ - torch.tensor(r["score"]) for r in rewards]    

    # Run PPO step.
    stats = ppo_trainer.step(promp_tensors, reply_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    if step % 20 == 0:
        print(f'objective/kl: {stats["objective/kl"]}')
        print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
        print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
        print('-'.join('' for x in range(100)))

1it [00:01,  1.83s/it]

objective/kl: 7.874390125274658
ppo/returns/mean: -0.386505126953125
ppo/policy/advantages_mean: -0.020643368363380432
---------------------------------------------------------------------------------------------------


10it [00:25,  2.92s/it]C:\Users\suppo\MLENV\Lib\site-packages\trl\trainer\ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -39.86 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
21it [00:59,  3.04s/it]

objective/kl: 16.69438934326172
ppo/returns/mean: -0.10692700743675232
ppo/policy/advantages_mean: 0.0515250563621521
---------------------------------------------------------------------------------------------------


23it [01:03,  2.53s/it]C:\Users\suppo\MLENV\Lib\site-packages\trl\trainer\ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -2.22 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
28it [01:15,  2.30s/it]C:\Users\suppo\MLENV\Lib\site-packages\trl\trainer\ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -1.25 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
41it [01:53,  3.42s/it]

...

2301it [1:41:43,  3.04s/it]

objective/kl: 11.228034973144531
ppo/returns/mean: -1.4084060192108154
ppo/policy/advantages_mean: -0.37908563017845154
---------------------------------------------------------------------------------------------------


2304it [1:41:51,  3.11s/it]C:\Users\suppo\MLENV\Lib\site-packages\trl\trainer\ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -2.13 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
2321it [1:42:27,  1.98s/it]

objective/kl: 4.705312252044678
ppo/returns/mean: -0.4427275061607361
ppo/policy/advantages_mean: 0.3497820496559143
---------------------------------------------------------------------------------------------------


2327it [1:42:40,  2.15s/it]C:\Users\suppo\MLENV\Lib\site-packages\trl\trainer\ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -4.79 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
2333it [1:42:57,  2.74s/it]C:\Users\suppo\MLENV\Lib\site-packages\trl\trainer\ppo_trainer.py:1308: UserWarning: KL divergence is starting to become negative: -4.02 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
2340it [1:43:21,  2.65s/it]


<a name='3.3'></a>
### 3.3 - Evaluate the Model Quantitatively

Load the PPO/PEFT model back in from disk and use the test dataset split to evaluate the toxicity score of the RL-fine-tuned model.

In [20]:
ppo_mean, ppo_std = evaluate_ner_generation(ppo_model, size=len(tokenized_datasets['test']))

print(f"Average need protect score after PPO: {ppo_mean} . Std: {ppo_std}")

521it [08:26,  1.03it/s]

Average need protect score after PPO: 0.33328257935858674 . Std: 0.4139923147111121


In [13]:
mean_improvement = (peft_mean - ppo_mean) / peft_mean
std_improvement = (peft_std - ppo_std) / peft_std

print(f'Percentage improvement of protect score after PPO:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

Percentage improvement of protect score after PPO:
mean: -44.48%
std: -13.55%


In [16]:
PPO_MODEL_PATH=f"./ppo_model_checkpoint"

ppo_model.save_pretrained(PPO_MODEL_PATH)
tokenizer.save_pretrained(PPO_MODEL_PATH)

('./ppo_model_checkpoint\\tokenizer_config.json',
 './ppo_model_checkpoint\\special_tokens_map.json',
 './ppo_model_checkpoint\\tokenizer.json')

In [24]:
index = 50

orginal_prompt = dataset['test'][index]['prompt']

def make_safe_prompt(prompt):
    return f"""Create a confidential prompt from the following prompt:

{prompt}

Prompt:"""

prompt = make_safe_prompt(orginal_prompt)

inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

peft_output = tokenizer.decode(
    ref_model.generate( input_ids=inputs["input_ids"], max_new_tokens=200)[0], 
    skip_special_tokens=True
)

ppo_output = tokenizer.decode(
    ppo_model.generate( input_ids=inputs["input_ids"], max_new_tokens=200)[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{orginal_prompt}')
print(dash_line)
print(f'MODEL GENERATION - PEFT MODEL:\n{peft_output}')
print(dash_line)
print(f'MODEL GENERATION - PPO MODEL:\n{ppo_output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
What are the most important events and festivals to attend in Rio de Janeiro, and how do they showcase the city's vibrant culture and traditions?
---------------------------------------------------------------------------------------------------
MODEL GENERATION - PEFT MODEL:
What are the most important events and festivals to attend in Rio de Janeiro, and how do they showcase the city's vibrant culture and traditions?
---------------------------------------------------------------------------------------------------
MODEL GENERATION - PPO MODEL:
What are the most important events and festivals to attend in Rio de Janeiro, and how do they showcase the city's vibrant culture and traditions?
